In [67]:
import os
import json
import pandas as pd

In [68]:
sorted(os.listdir())
data_path = '/Users/stevie/repos/language-app-frontend/assets/category_data/combined_data.json'
json_data = json.load(open(data_path, 'r'))

In [69]:
columns = [
    'category',
    'sub_category',
    'definition_en',
    'pronunciation',
    'singular_plural',
    'tense',
    'gender',
    'commonness',
    'part_of_speech',
    'formal_informal',
    'example_sentence_en',
    'example_sentence_it',
    'related_words_it',
    'article_type',
    'base_word',

    'combined_hash',
    'combined_audio_file',

    'english_text',
    'english_hash',
    'english_audio_file',
    'english_voice_name',
    'english_speaking_rate',
    'english_pitch',
    'english_duration_ms',

    'italian_text',
    'italian_hash',
    'italian_audio_file',
    'italian_voice_name',
    'italian_speaking_rate',
    'italian_pitch',
    'italian_duration_ms',
]

In [70]:
json_data['verb_directions']

[{'english': {'text': 'to turn',
   'hash': '93996043fafc7f77bfca89b3c8fd122054202f714313c70ea04116952b6411c9',
   'audio_file': './verb_directions/audio/individual/93996043fafc7f77bfca89b3c8fd122054202f714313c70ea04116952b6411c9.mp3',
   'voice_name': 'en-US-Wavenet-D',
   'speaking_rate': 1.0,
   'pitch': 0,
   'duration_ms': 888},
  'italian': {'text': 'girare',
   'hash': 'bf8afef44c03d2dadde4aeecef6e67adcb31870217811dcc3ac170b0ec31293d',
   'audio_file': './verb_directions/audio/individual/bf8afef44c03d2dadde4aeecef6e67adcb31870217811dcc3ac170b0ec31293d.mp3',
   'voice_name': 'it-IT-Wavenet-D',
   'speaking_rate': 0.75,
   'pitch': 0,
   'duration_ms': 1056},
  'category': 'verb',
  'sub_category': 'directions',
  'definition_en': 'change direction',
  'pronunciation': 'jee-RAH-reh',
  'singular_plural': '',
  'tense': 'infinitive',
  'gender': '',
  'commonness': 1,
  'part_of_speech': '',
  'formal_informal': '',
  'example_sentence_en': 'To reach the museum',
  'example_sentenc

In [71]:
import hashlib
data = {k: [] for k in columns}
for objs in json_data.values():
    for obj in objs:
        for key in obj.keys():
            if key not in ['italian', 'english']:
                data[key].append(obj[key])
        for lang in ['italian', 'english']:
            lang_obj = obj[lang]
            for key in lang_obj.keys():
                # if key in ['text', 'hash', 'audio_file', 'voice_name', 'speaking_rate', 'pitch', 'duration_ms']:
                data[f'{lang}_{key}'].append(lang_obj[key])
        
        param_str = f"{obj['italian']['hash']}_{obj['english']['hash']}"
        hash_object = hashlib.sha256(param_str.encode('utf-8'))
        hash_key = hash_object.hexdigest()
        # data['combined_hash'].append(hash_key)
        assert(data['combined_hash'][-1] == hash_key)
df = pd.DataFrame(data, columns=columns)

In [72]:
len(df)

1848

In [73]:
row = df.iloc[0]
print(row['italian_audio_file'])
print(row['english_audio_file'])
print(row['combined_audio_file'])
print(row['italian_audio_file'])
print(row['english_audio_file'])
print(row['combined_audio_file'])

./adjective_color/audio/individual/bc82cad0c5d879a22aa4c9138163b2803adb229167c40cf05800d055194e2455.mp3
./adjective_color/audio/individual/2916e2cecccb18c137b27e6fb9096c3afb5c4a622e538b87f7d1e8e4e08091d5.mp3
./adjective_color/audio/combined/0.mp3
./adjective_color/audio/individual/bc82cad0c5d879a22aa4c9138163b2803adb229167c40cf05800d055194e2455.mp3
./adjective_color/audio/individual/2916e2cecccb18c137b27e6fb9096c3afb5c4a622e538b87f7d1e8e4e08091d5.mp3
./adjective_color/audio/combined/0.mp3


In [80]:
df['sub_category'] = df['sub_category'].apply(lambda x: 'jewelry and metal working' if x == 'jewelry and metal workgin' else x)
df['sub_category'].value_counts()

sub_category
jewelry and metal working    466
directions                   217
shopping                     164
emotions                     160
general                       98
size                          68
restaurant                    60
food                          59
travel                        59
time                          57
people                        54
places                        51
weather                       50
drinks                        49
temperature                   44
fashion                       43
possessive                    24
quantity                      19
color                         18
cost                          17
quality                       16
emotion                       16
object                         9
demonstrative                  8
reflexive                      7
subject                        7
indefinite                     4
interrogative                  4
Name: count, dtype: int64

In [81]:
def fix_combined_audio_path(row):
    # category = path.split('/')[1]
    # filename = path.split('/')[-1]
    # path = f"category_data/audio/{category}/{filename}"
    category = row['category'].replace(' ', '_').lower()
    sub_category = row['sub_category'].replace(' ', '_').lower()
    path = f"category_data/audio/{category}_{sub_category}/{row['combined_hash']}.mp3"
    assert(os.path.exists(path)), f"Path does not exist: {path}"
    return path
# df['combined_audio_file'] = df['combined_audio_file'].apply(fix_combined_audio_path)
df['combined_audio_file'] = df.apply(fix_combined_audio_path, axis=1)

In [82]:
def fix_path(path):
    basename = os.path.basename(path)
    path = f"data/audio/{basename}"
    assert(os.path.exists(path)), f"Path does not exist: {path}"
    return path
df['italian_audio_file'] = df['italian_audio_file'].apply(fix_path)
df['english_audio_file'] = df['english_audio_file'].apply(fix_path)

In [83]:
from utils.s3_utils import upload_file

In [ ]:
# df.iloc[0]['combined_audio_file']
# import shutil
# for i, row in df.iterrows():
#     path = row['combined_audio_file']
#     assert(os.path.exists(path)), f"Path does not exist: {path}"
#     assert(row['combined_hash'] is not None), "Combined hash is None"
#     assert(row['combined_hash'] != ''), "Combined hash is empty"
#     assert('.mp3' in path), "Path does not contain .mp3"
#     dir = os.path.dirname(path)
#     new_path = os.path.join(dir, row['combined_hash'] + '.mp3')
#     shutil.move(path, new_path)
#     df.loc[i, 'combined_audio_file'] = new_path

In [ ]:
from tqdm import tqdm
for i, row in tqdm(df.iterrows()):
    for key in ['combined_audio_file', 'italian_audio_file', 'english_audio_file']:
        assert(os.path.exists(row[key])), f"Path does not exist: {row[key]}"
        success = upload_file(row[key])
        if not success:
            print(f"Failed to upload {row[key]}")
df.to_csv('category_data/combined_data.csv', index=False)